<a href="https://colab.research.google.com/github/GLOMQuyet/symato/blob/main/DataMaker_for_Alpaca_style_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.1 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/tatsu-lab/stanford_alpaca.git

Cloning into 'stanford_alpaca'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 103 (delta 0), reused 1 (delta 0), pack-reused 100
Receiving objects: 100% (103/103), 9.13 MiB | 10.79 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
%cd stanford_alpaca

/content/stanford_alpaca


## Alpaca Data Creator


<img src="https://github.com/tatsu-lab/stanford_alpaca/blob/main/assets/parse_analysis.png?raw=1" alt="example image" width="600">

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2

In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import openai

openai.api_key =''
os.environ['OPENAI_API_KEY'] = ''

## Data Generations

In [ ]:
import time
import json
import os
import random
import re
import string
from functools import partial
from multiprocessing import Pool

import numpy as np
import tqdm
from rouge_score import rouge_scorer
import utils

import fire

In [ ]:

def encode_prompt(prompt_instructions):
    """Encode multiple prompt instructions into a single string."""
    prompt = open("./prompt.txt").read() + "\n"

    for idx, task_dict in enumerate(prompt_instructions):
        (instruction, input, output) = task_dict["instruction"], task_dict["input"], task_dict["output"]
        instruction = re.sub(r"\s+", " ", instruction).strip().rstrip(":")
        input = "<noinput>" if input.lower() == "" else input
        prompt += f"###\n"
        prompt += f"{idx + 1}. Instruction: {instruction}\n"
        prompt += f"{idx + 1}. Input:\n{input}\n"
        prompt += f"{idx + 1}. Output:\n{output}\n"
    prompt += f"###\n"
    prompt += f"{idx + 2}. Instruction:"
    return prompt


def post_process_gpt3_response(num_prompt_instructions, response):
    if response is None:
        return []
    raw_instructions = f"{num_prompt_instructions+1}. Instruction:" + response["text"]
    raw_instructions = re.split("###", raw_instructions)
    instructions = []
    for idx, inst in enumerate(raw_instructions):
        # if the decoding stops due to length, the last example is likely truncated so we discard it
        if idx == len(raw_instructions) - 1 and response["finish_reason"] == "length":
            continue
        idx += num_prompt_instructions + 1
        splitted_data = re.split(f"{idx}\.\s+(Instruction|Input|Output):", inst)
        if len(splitted_data) != 7:
            continue
        else:
            inst = splitted_data[2].strip()
            input = splitted_data[4].strip()
            input = "" if input.lower() == "<noinput>" else input
            output = splitted_data[6].strip()
        # filter out too short or too long instructions
        if len(inst.split()) <= 3 or len(inst.split()) > 150:
            continue
        # filter based on keywords that are not suitable for language models.
        blacklist = [
            "image",
            "images",
            "graph",
            "graphs",
            "picture",
            "pictures",
            "file",
            "files",
            "map",
            "maps",
            "draw",
            "plot",
            "go to",
            "video",
            "audio",
            "music",
            "flowchart",
            "diagram",
        ]
        blacklist += []
        if any(find_word_in_string(word, inst) for word in blacklist):
            continue
        # We found that the model tends to add "write a program" to some existing instructions, which lead to a lot of such instructions.
        # And it's a bit comfusing whether the model need to write a program or directly output the result.
        # Here we filter them out.
        # Note this is not a comprehensive filtering for all programming instructions.
        if inst.startswith("Write a program"):
            continue
        # filter those starting with punctuation
        if inst[0] in string.punctuation:
            continue
        # filter those starting with non-english character
        if not inst[0].isascii():
            continue
        instructions.append({"instruction": inst, "input": input, "output": output})
    return instructions


def find_word_in_string(w, s):
    return re.compile(r"\b({0})\b".format(w), flags=re.IGNORECASE).search(s)


def generate_instruction_following_data(
    output_dir="./",
    seed_tasks_path="./seed_tasks.jsonl",
    num_instructions_to_generate=10,
    model_name="text-davinci-003",
    num_prompt_instructions=3,
    request_batch_size=5,
    temperature=1.0,
    top_p=1.0,
    num_cpus=16,
):
    seed_tasks = [json.loads(l) for l in open(seed_tasks_path, "r")]
    seed_instruction_data = [
        {"instruction": t["instruction"], "input": t["instances"][0]["input"], "output": t["instances"][0]["output"]}
        for t in seed_tasks
    ]
    print(f"Loaded {len(seed_instruction_data)} human-written seed instructions")

    os.makedirs(output_dir, exist_ok=True)
    request_idx = 0
    # load the LM-generated instructions
    machine_instruction_data = []
    if os.path.exists(os.path.join(output_dir, "regen.json")):
        machine_instruction_data = utils.jload(os.path.join(output_dir, "regen.json"))
        print(f"Loaded {len(machine_instruction_data)} machine-generated instructions")

    # similarities = {}
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)

    # now let's generate new instructions!
    progress_bar = tqdm.tqdm(total=num_instructions_to_generate)
    if machine_instruction_data:
        progress_bar.update(len(machine_instruction_data))

    # first we tokenize all the seed instructions and generated machine instructions
    all_instructions = [d["instruction"] for d in seed_instruction_data] + [
        d["instruction"] for d in machine_instruction_data
    ]
    all_instruction_tokens = [scorer._tokenizer.tokenize(inst) for inst in all_instructions]

    while len(machine_instruction_data) < num_instructions_to_generate:
        request_idx += 1

        batch_inputs = []
        for _ in range(request_batch_size):
            # only sampling from the seed tasks
            prompt_instructions = random.sample(seed_instruction_data, num_prompt_instructions)
            prompt = encode_prompt(prompt_instructions)
            batch_inputs.append(prompt)
        decoding_args = utils.OpenAIDecodingArguments(
            temperature=temperature,
            n=1,
            max_tokens=3072,  # hard-code to maximize the length. the requests will be automatically adjusted
            top_p=top_p,
            stop=["\n20", "20.", "20."],
        )
        request_start = time.time()
        results = utils.openai_completion(
            prompts=batch_inputs,
            model_name=model_name,
            batch_size=request_batch_size,
            decoding_args=decoding_args,
            logit_bias={"50256": -100},  # prevent the <|endoftext|> token from being generated
        )
        request_duration = time.time() - request_start

        process_start = time.time()
        instruction_data = []
        for result in results:
            new_instructions = post_process_gpt3_response(num_prompt_instructions, result)
            instruction_data += new_instructions

        total = len(instruction_data)
        keep = 0
        for instruction_data_entry in instruction_data:
            # computing similarity with the pre-tokenzied instructions
            new_instruction_tokens = scorer._tokenizer.tokenize(instruction_data_entry["instruction"])
            with Pool(num_cpus) as p:
                rouge_scores = p.map(
                    partial(rouge_scorer._score_lcs, new_instruction_tokens),
                    all_instruction_tokens,
                )
            rouge_scores = [score.fmeasure for score in rouge_scores]
            most_similar_instructions = {
                all_instructions[i]: rouge_scores[i] for i in np.argsort(rouge_scores)[-10:][::-1]
            }
            if max(rouge_scores) > 0.7:
                continue
            else:
                keep += 1
            instruction_data_entry["most_similar_instructions"] = most_similar_instructions
            instruction_data_entry["avg_similarity_score"] = float(np.mean(rouge_scores))
            machine_instruction_data.append(instruction_data_entry)
            all_instructions.append(instruction_data_entry["instruction"])
            all_instruction_tokens.append(new_instruction_tokens)
            progress_bar.update(1)
        process_duration = time.time() - process_start
        print(f"Request {request_idx} took {request_duration:.2f}s, processing took {process_duration:.2f}s")
        print(f"Generated {total} instructions, kept {keep} instructions")
        utils.jdump(machine_instruction_data, os.path.join(output_dir, "regen.json"))


## the data

In [ ]:
json_new_tasks = '''
[
    {
        "id": "new_seed_task_3",
        "name": "lựa chọn bữa tối chay",
        "instruction": "Một số lựa chọn bữa tối chay có hàm lượng protein cao là gì?",
        "instances": [{"input": "",
                      "output": "Một số lựa chọn bữa tối chay có hàm lượng protein cao bao gồm súp đậu lăng, cà ri đậu xanh, đậu phụ xào, tacos tempeh và ớt nhồi quinoa."}],
        "is_classification": false
    },
    {
        "id": "new_seed_task_4",
        "name": "giải thích chính sách hoàn trả",
        "instruction": "Can you explain your refund policy?",
        "instances": [{"input": "",
                      "output": "Our refund policy allows customers to request a refund within 30 days of purchase if they are unsatisfied with the product or service. To be eligible for a refund, the product must be in its original condition and packaging, and services must not have been completed."}],
        "is_classification": false
    },
    {
        "id": "new_seed_task_5",
        "name": "khắc phục sự cố kết nối wifi",
        "instruction": "Tôi đang gặp sự cố khi kết nối với mạng Wi-Fi của mình. Tôi nên làm gì?",
        "instances": [{"input": "",
                      "output": "Trước tiên, hãy kiểm tra xem thiết bị của bạn có nằm trong phạm vi phủ sóng của bộ định tuyến Wi-Fi không. Sau đó, xác minh rằng bạn đã nhập đúng tên mạng và mật khẩu. Nếu sự cố vẫn tiếp diễn, hãy thử khởi động lại cả thiết bị và bộ định tuyến của bạn. Nếu bạn vẫn không thể kết nối, hãy liên hệ với nhà cung cấp dịch vụ internet của bạn để được hỗ trợ thêm."}],
        "is_classification": false
    },
    {
        "id": "new_seed_task_6",
        "name": "hướng dẫn thay đổi mật khẩu",
        "instruction": "Làm thế nào để thay đổi mật khẩu của tôi?",
        "instances": [{"input": "",
                      "output": "Để thay đổi mật khẩu của bạn, hãy đăng nhập vào tài khoản của bạn, đi tới phần cài đặt hoặc quản lý tài khoản và tìm tùy chọn thay đổi mật khẩu của bạn. Nhập mật khẩu hiện tại của bạn, sau đó nhập mật khẩu mới của bạn hai lần để xác nhận. Lưu các thay đổi của bạn để cập nhật mật khẩu của bạn."}],
        "is_classification": false
    },
    {
        "id": "new_seed_task_7",
        "name": "yêu cầu gói bị mất",
        "instruction": "Gói hàng của tôi vẫn chưa đến. Tôi nên làm gì?",
        "instances": [{"input": "",
                      "output": "Trước tiên, hãy kiểm tra email hoặc tài khoản xác nhận đơn đặt hàng của bạn để biết số theo dõi và ước tính ngày giao hàng. Nếu đã qua ngày giao hàng, hãy liên hệ với hãng vận chuyển kèm theo số theo dõi của bạn để cập nhật. Nếu nhà cung cấp dịch vụ không thể giải quyết vấn đề, vui lòng liên hệ với nhóm hỗ trợ khách hàng của chúng tôi để được hỗ trợ."}],
        "is_classification": false
    },
    {
        "id": "new_seed_task_8",
        "name": "thay đổi thứ tự",
        "instruction": "Tôi cần sửa đổi đơn đặt hàng của mình. Làm thế nào tôi có thể làm điều đó?",
        "instances": [{"input": "",
                      "output": "Để sửa đổi đơn đặt hàng của bạn, vui lòng liên hệ với nhóm hỗ trợ khách hàng của chúng tôi càng sớm càng tốt với số đơn đặt hàng của bạn và những thay đổi bạn muốn thực hiện. Xin lưu ý rằng có thể không sửa đổi được nếu đơn đặt hàng đã được xử lý hoặc vận chuyển."}],
        "is_classification": false
    }
]
'''


In [ ]:
new_seed_tasks = json.loads(json_new_tasks)
print(new_seed_tasks)


[{'id': 'new_seed_task_3', 'name': 'lựa chọn bữa tối chay', 'instruction': 'Một số lựa chọn bữa tối chay có hàm lượng protein cao là gì?', 'instances': [{'input': '', 'output': 'Một số lựa chọn bữa tối chay có hàm lượng protein cao bao gồm súp đậu lăng, cà ri đậu xanh, đậu phụ xào, tacos tempeh và ớt nhồi quinoa.'}], 'is_classification': False}, {'id': 'new_seed_task_4', 'name': 'giải thích chính sách hoàn trả', 'instruction': 'Can you explain your refund policy?', 'instances': [{'input': '', 'output': 'Our refund policy allows customers to request a refund within 30 days of purchase if they are unsatisfied with the product or service. To be eligible for a refund, the product must be in its original condition and packaging, and services must not have been completed.'}], 'is_classification': False}, {'id': 'new_seed_task_5', 'name': 'khắc phục sự cố kết nối wifi', 'instruction': 'Tôi đang gặp sự cố khi kết nối với mạng Wi-Fi của mình. Tôi nên làm gì?', 'instances': [{'input': '', 'outpu

In [ ]:
with open('new_seed_tasks.jsonl', 'w') as outfile:
    for task_dict in new_seed_tasks:
        json.dump(task_dict, outfile)
        outfile.write('\n')

In [ ]:
!mkdir new_tasks

In [ ]:
generate_instruction_following_data(
    output_dir="./new_tasks/",
    seed_tasks_path="./new_seed_tasks.jsonl",
    num_instructions_to_generate=5,
    model_name="text-davinci-003",
    num_prompt_instructions=3,
    request_batch_size=2,
    temperature=1.0,
    top_p=1.0,
    num_cpus=4,
)

Loaded 6 human-written seed instructions
Loaded 0 machine-generated instructions




  0%|          | 0/5 [00:00<?, ?it/s]


  0%|          | 0/5 [05:11<?, ?it/s]



prompt_batches: 100%|██████████| 1/1 [00:35<00:00, 35.70s/it]


 20%|██        | 1/5 [00:35<02:23, 35.78s/it]

 60%|██████    | 3/5 [00:35<00:18,  9.32s/it]

Request 1 took 35.71s, processing took 0.25s
Generated 4 instructions, kept 4 instructions


prompt_batches:   0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:OpenAIError: This model's maximum context length is 4097 tokens, however you requested 5315 tokens (2243 in your prompt; 3072 for the completion). Please reduce your prompt; or completion length..


prompt_batches: 100%|██████████| 1/1 [00:28<00:00, 28.63s/it]


100%|██████████| 5/5 [01:04<00:00, 11.92s/it]

8it [01:04,  5.70s/it]                       

10it [01:04,  6.49s/it]

Request 2 took 28.63s, processing took 0.32s
Generated 6 instructions, kept 6 instructions


In [ ]:
import json

file_path = "/content/stanford_alpaca_vietnamese/new_tasks/regen.json"

with open(file_path, 'r') as json_file:
    data = json.load(json_file)

# drop the most similar as measured by Rouge
for dictionary in data:
    dictionary.pop('most_similar_instructions', None)
    dictionary.pop('avg_similarity_score',None)


print(data)


[{'instruction': 'Tôi cần biết thêm về ứng dụng của chúng ta. Tôi nên làm gì?', 'input': '', 'output': 'Hãy truy cập trang web của chúng tôi để xem thêm thông tin về ứng dụng của chúng tôi. Bạn cũng có thể tham khảo những tài liệu hướng dẫn và hướng dẫn sử dụng ứng dụng của chúng tôi để biết thêm thông tin. Bạn cũng có thể xem thêm các video hướng dẫn về cách sử dụng tính năng và công cụ của ứng dụng của chúng tôi. Nếu bạn cần thêm trợ giúp, vui lòng liên hệ với dịch vụ hỗ trợ khách hàng của chúng tôi để được hỗ trợ.'}, {'instruction': 'Tôi muốn biết cách đặt một cuộc hẹn. Tôi nên làm gì?', 'input': '', 'output': 'Để đặt một cuộc hẹn, hãy tìm kiếm qua trang web của chúng tôi hoặc gọi điện vàoở địa chỉ mà chúng tôi cung cấp. Nếu bạn muốn tự đặt lịch hẹn trên trang web của chúng tôi, hãy chọn ngày và giờ mà bạn muốn dịch vụ và điền vào các thông tin cần thiết. Sau khi hoàn thành đặt hẹn, hãy nhận được một email xác nhận hoặc một số điện thoại để xác nhận cuộc hẹn của bạn.'}, {'instructio

In [ ]:
len(data)

10

In [ ]:
for i in range(10):
  print(data[i])

{'instruction': 'Tôi cần biết thêm về ứng dụng của chúng ta. Tôi nên làm gì?', 'input': '', 'output': 'Hãy truy cập trang web của chúng tôi để xem thêm thông tin về ứng dụng của chúng tôi. Bạn cũng có thể tham khảo những tài liệu hướng dẫn và hướng dẫn sử dụng ứng dụng của chúng tôi để biết thêm thông tin. Bạn cũng có thể xem thêm các video hướng dẫn về cách sử dụng tính năng và công cụ của ứng dụng của chúng tôi. Nếu bạn cần thêm trợ giúp, vui lòng liên hệ với dịch vụ hỗ trợ khách hàng của chúng tôi để được hỗ trợ.'}
{'instruction': 'Tôi muốn biết cách đặt một cuộc hẹn. Tôi nên làm gì?', 'input': '', 'output': 'Để đặt một cuộc hẹn, hãy tìm kiếm qua trang web của chúng tôi hoặc gọi điện vàoở địa chỉ mà chúng tôi cung cấp. Nếu bạn muốn tự đặt lịch hẹn trên trang web của chúng tôi, hãy chọn ngày và giờ mà bạn muốn dịch vụ và điền vào các thông tin cần thiết. Sau khi hoàn thành đặt hẹn, hãy nhận được một email xác nhận hoặc một số điện thoại để xác nhận cuộc hẹn của bạn.'}
{'instruction':